In [208]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [209]:
tag_users = pd.read_csv("tag_users.csv")
tag_questions = pd.read_csv("tag_questions.csv")
tags = pd.read_csv("tags.csv")
professionals = pd.read_csv("professionals.csv")
answers = pd.read_csv("answers.csv")
emails = pd.read_csv("emails.csv")
questions = pd.read_csv("questions.csv")
matches = pd.read_csv("matches.csv")
answers_scores = pd.read_csv("answer_scores.csv")
group_members = pd.read_csv("group_memberships.csv")
schools_members = pd.read_csv("school_memberships.csv")

In [210]:
def get_user_tags(user_id):
    user_tags = tag_users[tag_users["tag_users_user_id"] == user_id]["tag_users_tag_id"]
    tags_names =tags.loc[tags["tags_tag_id"].isin(user_tags)]
    return(tags)

In [211]:
def get_user_answers(user_id):
    all_answers = answers.loc[answers["answers_author_id"]==user_id]
    return all_answers

In [212]:
def match_email_question(email_date):
    f = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S UTC+0000")
    b  =f(email_date)
    t = questions["questions_date_added"].apply(f)
    return questions.iloc[(b-t).abs().argsort()[:1]]

In [213]:
immediate_emails = emails[emails["emails_frequency_level"]=="email_notification_immediate"]
immediate_matches = matches[matches["matches_email_id"].isin(immediate_emails["emails_id"])]
immediate_emails = immediate_emails[immediate_emails["emails_id"].isin(matches["matches_email_id"])]

immediate_emails = immediate_emails.rename(columns={'emails_id':"id"})
immediate_matches = immediate_matches.rename(columns={'matches_email_id':"id"})

In [214]:
emails_questions = pd.merge(immediate_emails, immediate_matches, how="left", on="id")
emails_questions = emails_questions.rename(columns = {"matches_question_id": "question_id", "emails_recipient_id": "professional_id"})
answers = answers.rename(columns = {"answers_question_id": "question_id", "answers_author_id": "professional_id"})


In [215]:
full_data = pd.merge(emails_questions, answers,how='left',  on=["professional_id","question_id" ])
target = full_data['answers_id'].fillna(0)
target = pd.to_numeric(target, errors='coerce').fillna(1).astype(int)
full_data["q_answered"] = target
full_data = full_data.rename(columns = {"id": "email_id"})
date_vectorizer = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S UTC+0000")
full_data["emails_date_sent"] = full_data["emails_date_sent"].apply(date_vectorizer)
indices = full_data["answers_date_added"][full_data["answers_date_added"].notnull()].index.values
full_data.loc[indices, "answers_date_added"] = full_data["answers_date_added"][full_data["answers_date_added"].notnull()].apply(date_vectorizer)
full_data["time_taken"] =  pd.to_datetime(full_data["answers_date_added"]) - pd.to_datetime(full_data["emails_date_sent"])
indices2 = full_data["time_taken"][full_data["time_taken"] < datetime.timedelta( days=0)].index.values
full_data.loc[indices2, "time_taken"]=datetime.timedelta(days=0, seconds=0,minutes=0, hours=0 )

In [216]:
response = full_data.groupby("professional_id").mean().drop(columns = ["email_id"]).reset_index()
professionals_dataset = pd.DataFrame(columns = ["professional_id"])
professionals_dataset["professional_id"] = professionals["professionals_id"]
professionals_dataset = pd.merge(professionals_dataset, response,how='left',  on=["professional_id" ])
professionals_dataset = professionals_dataset.rename(columns = {"q_answered": "response_rate"})
grouped = full_data.groupby('professional_id')["time_taken"]
time_mean = grouped.apply(lambda x: np.mean(x))
professionals_dataset = pd.merge(professionals_dataset, time_mean,how='left',  on=["professional_id" ])
professionals_dataset=  professionals_dataset.rename(columns ={"time_taken": "avg_time_taken"})
answers_count = answers["professional_id"].value_counts().reset_index().rename(columns = {"professional_id":"number_q_answered","index":"professional_id",  })
professionals_dataset = pd.merge(professionals_dataset, answers_count,how='left',  on=["professional_id" ])
all_tags = pd.merge(tag_users.rename(columns = {"tag_users_tag_id": "tag_id"}),tags.rename(columns = {"tags_tag_id": "tag_id"}),how='left',  on=["tag_id" ])
foll_tags = all_tags.groupby('tag_users_user_id')['tags_tag_name'].apply(list).reset_index(name='following_tags').rename(columns = {"tag_users_user_id": "professional_id"})
professionals_dataset = pd.merge(professionals_dataset, foll_tags,how='left',  on=["professional_id" ])
a = pd.merge(answers["question_id"].reset_index(), tag_questions.rename(columns = {"tag_questions_question_id": "question_id"}),how='left',  on=["question_id" ])
b = pd.merge(a.rename(columns = {"tag_questions_tag_id":"tags_tag_id"}), tags,how='left',  on=["tags_tag_id" ]).drop(columns = ["index", "tags_tag_id"])
c = pd.merge(b, answers,how='left',  on=["question_id" ]).drop(columns = ["answers_id", "answers_date_added", "answers_body", "question_id"])
d = c.groupby("professional_id")["tags_tag_name"].apply(list).reset_index()
professionals_dataset = pd.merge(professionals_dataset, d.rename(columns = {"tags_tag_name": "prev_q_tags"}),how='left',  on=["professional_id" ])
prof_score = pd.merge(answers[["professional_id","answers_id" ]], answers_scores.rename(columns = {"id":"answers_id"}),how='left',  on=["answers_id" ]).drop(columns = ["answers_id"])
prof_score.groupby("professional_id").mean().reset_index()
professionals_dataset = pd.merge(professionals_dataset,prof_score,how='left',  on=["professional_id" ]).rename(columns = {"score":"avg_ansrs_score"})
prof_grp = group_members["group_memberships_user_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "group_memberships_user_id":"num_groups"})
professionals_dataset = pd.merge(professionals_dataset,prof_grp,how='left',  on=["professional_id" ])
prof_schl = schools_members["school_memberships_user_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "school_memberships_user_id":"num_schools"})
professionals_dataset = pd.merge(professionals_dataset,prof_schl,how='left',  on=["professional_id" ])
prof_emls = immediate_emails["emails_recipient_id"].value_counts().reset_index().rename(columns = {"index":"professional_id", "emails_recipient_id":"num_emails"})
professionals_dataset = pd.merge(professionals_dataset,prof_emls,how='left',  on=["professional_id" ])
professionals_dataset["answrs_emails_ratio"] = professionals_dataset["number_q_answered"] / professionals_dataset["num_emails"]


In [217]:
professionals_dataset.head(1)

,professional_id,response_rate,avg_time_taken,number_q_answered,following_tags,prev_q_tags,avg_ansrs_score,num_groups,num_schools,num_emails,answrs_emails_ratio
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaT,1.0,NaN,"[consulting, resume, consulting, resume, consu...",5.0,NaN,NaN,NaN,NaN


In [218]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.utils import shuffle
import itertools
import shutil
from bs4 import BeautifulSoup
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from dateutil import parser
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def lowercase(input):
  return input.lower()

def remove_punctuation(input):
  return input.translate(str.maketrans('','', string.punctuation))

def remove_whitespaces(input):
  return " ".join(input.split())
  
def remove_html_tags(input):
    soup = BeautifulSoup(input, "html.parser")
    stripped_input = soup.get_text(separator=" ")
    return stripped_input

def tokenize(input):
  return word_tokenize(input)

def remove_stop_words(input):
  input = word_tokenize(input)
  return [word for word in input if word not in stopwords.words('english')]

def lemmatize(input):
  lemmatizer=WordNetLemmatizer()
  input_str=word_tokenize(input)
  new_words = []
  for word in input_str:
    new_words.append(lemmatizer.lemmatize(word))
  return ' '.join(new_words)

#Function that calls all other functions together to perform NLP on a given text
def nlp_pipeline(input):
  return lemmatize(' '.join(remove_stop_words(remove_whitespaces(remove_punctuation(remove_html_tags(lowercase(input)))))))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

#Turn tags into a set for faster checking of whether a tag exists or not
unique_tags = set(tags['tags_tag_name'])

def find_topics(question_body):
  """
  Function that takes a question as an input, and finds the two most important topics/tags
  If the found topics exist in the already existing database of tags, we add these tags
  to the professional who answered the question
  """

  text = nlp_pipeline(question_body)
  count_vectorizer = CountVectorizer(stop_words='english')
  count_data = count_vectorizer.fit_transform([text])
  # One topic that has an avg of two words because most questions had 1/2 tags
  number_topics = 1
  number_words = 2
  # Create and fit the LDA model
  lda = LDA(n_components=number_topics, n_jobs=-1)
  lda.fit(count_data)

  words = count_vectorizer.get_feature_names()

  #Get topics from model. They are represented as a list e.g. ['military','army']
  topics = [[words[i] for i in topic.argsort()[:-number_words - 1:-1]] for (topic_idx, topic) in enumerate(lda.components_)]
  topics = np.array(topics).ravel()
  #Only use topics for which a tag already exists
  existing_topics = []
  for tag in topics:
    if tag in unique_tags:
      existing_topics.append(tag)

  return existing_topics

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gaber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gaber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [219]:
question_body = "Hello, can you please give me advice on which courses to take to have a career in finance?"
find_topics(question_body)

['hello', 'finance']

In [220]:
question_body = questions["questions_body"][2]
print(question_body)
find_topics(question_body)

I'm planning on going abroad for my first job. It will be a teaching job and I don't have any serious career ideas. I don't know what job I would be working if I stay home instead so I'm assuming staying or leaving won't makeba huge difference in what I care about, unless I find something before my first job. I can think of ways that going abroad can be seen as good and bad. I do not know which side respectable employers willl side with. #working-abroad #employment- #overseas


['job', 'know']

In [221]:
# a toy function that is fast:
question_body = "Hello, can you please give me advice on which courses to take to have a career in finance?"
def find_topics_toy(question_body):
    return question_body.split(" ")[0]

In [222]:
# let's get the answers' authors first:
my_answers_authors = answers.drop(columns = ["answers_date_added", "answers_body"])
my_answers_authors.head(1)

,answers_id,professional_id,question_id
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54


In [223]:
# let's now get the questions_body with the question id:
my_questions_body = questions[["questions_id", "questions_body"]].rename(columns = {"questions_id":"question_id"})
my_questions_body.head(1)

,question_id,questions_body
0,332a511f1569444485cf7a7a556a5e54,What is a maths teacher? what is a ma...


In [224]:
# let's merge these together:
add_them = pd.merge(my_answers_authors,my_questions_body,how='left',  on=["question_id" ])
add_them.head(1)

,answers_id,professional_id,question_id,questions_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,What is a maths teacher? what is a ma...


In [225]:
# let's drop the columns that we don't need:
our_final_data = add_them.drop(columns = ["answers_id", "question_id"])
our_final_data.head(1)

,professional_id,questions_body
0,36ff3b3666df400f956f8335cf53e09e,What is a maths teacher? what is a ma...


In [226]:
# let's apply the function and get our new tags:
new_tags= our_final_data["questions_body"].apply(find_topics_toy).values

# let's add it as a column:
our_final_data["new_tags"] = new_tags

In [227]:
# we have a small issue, the professionals in that dataset are not unique because they answered different questions and
# from each question, they get a unique new two tags. So let's group the tags of the same professionals together:

In [228]:
#group the tags of the same professionals in a list (this will be many tags for each professionl because 
# for each question they answer, they get two new tags)
our_final_data = our_final_data.groupby("professional_id")["new_tags"].apply(list).reset_index()

In [229]:
# finally, let's merge the new tags with the prev tags we had:
toy = pd.merge(professionals_dataset,our_final_data,how='left',  on=["professional_id" ])
professionals_dataset["prev_q_tags"] = toy["prev_q_tags"]+toy["new_tags"]

In [230]:
# final data: 
professionals_dataset

,professional_id,response_rate,avg_time_taken,number_q_answered,following_tags,prev_q_tags,avg_ansrs_score,num_groups,num_schools,num_emails,answrs_emails_ratio
0,9ced4ce7519049c0944147afb75a8ce3,NaN,NaT,1.0,NaN,"[consulting, resume, consulting, resume, consu...",5.0,NaN,NaN,NaN,NaN
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0c673e046d824ec0ad0ebe012a0673e4,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",1.0,NaN,NaN,70.0,0.557143
3,0c673e046d824ec0ad0ebe012a0673e4,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",0.0,NaN,NaN,70.0,0.557143
4,0c673e046d824ec0ad0ebe012a0673e4,0.0,NaT,39.0,"[consulting, education, consulting, education,...","[experience, healthcare, doctor, anesthesiolog...",3.0,NaN,NaN,70.0,0.557143
...,...,...,...,...,...,...,...,...,...,...,...
68186,9f267950ab8e43e2afcf564327049297,NaN,NaT,NaN,[broadcast-media],NaN,NaN,NaN,NaN,NaN,NaN
68187,4a7e1e5dd884488da283e1cab6ad11e0,NaN,NaT,NaN,"[telecommunications, firstnet, #inbuildingsolu...",NaN,NaN,NaN,NaN,NaN,NaN
68188,ea75c5fce38348e0a151c3c346929e6a,NaN,NaT,NaN,"[computer-software, electrical-engineering, ha...",NaN,NaN,NaN,NaN,NaN,NaN
68189,34f6222c3c02480ca2df8a3e4ba878d0,NaN,NaT,NaN,[telecommunications],NaN,NaN,NaN,NaN,NaN,NaN
